In [ ]:
#def ClassificationEmotion(seq_tar_im, training_set, path_for_pca, path_for_som, lr, sigma,  test_mode = False):
#  training_features = get_features(training_set, path_for_pca, test= False)
  #shape_pca = path_for_pca + '/shape_pca'
  #structure_pca = path_for_pca + '/structure_pca'
  #cartoon_pca = path_for_pca + '/cartoon_pca'
  #texture_pca = path_for_pca + '/texture_pca'
#  expression_map = get_expression_map(training_features, training_set, path_for_som, lr, sigma)
#  features = get_features(seq_tar_im, path_for_pca, test = True)




#  target_emotion = 0
#  return target_emotion

Creo que lo que realmente necesito es un clase classifier o expresion map que dado training guarde los pca (y los pueda cargar si quiero),
Dado que luego convoque a la clase con un conjunto de prueba o secuencia y este me deje usar los pca dados y luego classificar. Además va a tener funciones de testing

La clase se crea y luego se debe cargar antes de usar. Puede cargarse  los atributos o calcularos a partir de un conjunto de entrenamiento.

Luego de cargarse su fuincionalidad es de poder tanto usarse para medir la precisión como para obtener respuesta, asi como poder hacerlo con los datos de entrenamiento. o entrenar nuevos.

Ademas debe dejar guardado las cosas en los path especificados. LAs más importantes son las imagenes alineadas y las alineadas + croppeo (y las cartoon + textura). y los pca (5 destinos)
Cuando se hace test se debe dejar tambien los path, todos menos los de los PCA.

Deberia ya que estoy seguir el histograma de una imagen en cada paso. Para ver que están haciendo las operaciones.



In [1]:
%matplotlib inline

import numpy as np # NUMPY
import pandas as pd # PANDAS
import matplotlib.pyplot as plt # MATPLOTLIB
import cv2
import os

!git clone https://github.com/aliciaamaliaalvarezmon/SynthesisEmotions.git
%run /content/SynthesisEmotions/Notebook/auxfunctions.ipynb

ModuleNotFoundError: No module named 'cv2'

In [ ]:


%run /content/SynthesisEmotions/universityfiles/ExpressionMapFunctionsAux.ipynb
%run /content/SynthesisEmotions/universityfiles/SOM2ver.ipynb

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pickle as pk

def doPCA(data, comp):
    std_scaler = StandardScaler()
    scaled_landDiff = std_scaler.fit_transform(data)
    pca = PCA(n_components=comp)
    pca.fit_transform(scaled_landDiff)
    b_shape = pca.fit_transform(scaled_landDiff)
    print(sum(pca.explained_variance_ratio_))
    print(b_shape.shape)
    return b_shape, pca

In [ ]:
def align_and_transform_datavector_of_images(reference_image, reference_landmarks, landmarks, images, pathi):
    transformed_landmarks_total = []
    for i in range(0, len(landmarks)):

        tr_Y_img, Z_pts = align_and_transform_image(reference_image, reference_landmarks, landmarks.iloc[i], images[i])
        transformed_landmarks_total.append(Z_pts)
        filename = 'warped_image_procustres_'+ str(images[i].split('/')[8])
        path = pathi
        cv2.imwrite(os.path.join(path , filename), tr_Y_img)
    transformed_landmarks_total = np.asarray(transformed_landmarks_total)
    return transformed_landmarks_total

In [ ]:
import numpy as np

class EmotionClassifier():
    """
    The 2-D, rectangular grid self-organizing map class using Numpy.
    """
    def __init__(self, training_data_images, training_data_landmarks, aligned_training_images_path, aligned_training_landmarks_path, shape_diff_path, test_data_images, test_data_landmarks,  m=9, n=9, lr=0.25, sigma=1.75):


        # Initialize descriptive features of SOM
        self.m = m
        self.n = n
        self.shape = (m, n)
        self.lr = lr
        self.sigma = sigma

        self.training_set_img_folder = training_data_images
        self.test_set_img_folder = test_data_images
        self.training_set_landmarks_folder = training_data_landmarks
        self.test_set_landmarks_folder = test_data_landmarks

        self.aligned_training_images_path = aligned_training_images_path
        self.aligned_training_landmarks_path = aligned_training_landmarks_path
        self.shape_diff_path = shape_diff_path
        #self.aligned_test_images_path = aligned_test_images_path
        #self.aligned_test_landmarks_path = aligned_test_landmarks_path

        # Initialize weights
        self.shape_pca = None
        self.structure_pca = None
        self.cartoon_pca = None
        self.texture_pca = None

        self.shape_scaler = None
        self.structure_scaler = None
        self.cartoon_scaler = None
        self.texture_scaler = None

        self.training_set_features = None
        self.test_set_features = None



        # Set after fitting


    def _change_training_img_loc(self, loc):
        """
        Return the indices of an m by n array.
        """
        self.training_set_img_folder = loc
        return self.training_set_img_folder

    def _change_test_img_loc(self, loc):
        """
        Return the indices of an m by n array.
        """
        self.test_set_img_folder = loc
        return self.test_set_img_folder

    def _change_training_lan_loc(self, loc):
        """
        Return the indices of an m by n array.
        """
        self.training_set_landmarks_folder = loc
        return self.training_set_landmarks_folder

    def _change_test_lan_loc(self, loc):
        """
        Return the indices of an m by n array.
        """
        self.test_set_landmarks_folder = loc
        return self.test_set_landmarks_folder

    def _save_shape_pca(self):
      pk.dump(self.shape_pca, open("shape_pca.pkl","wb"))

    def _load_shape_pca(self):
      self.shape_pca = pk.load(open("shape_pca.pkl","rb"))
      return self.shape_pca






    def _get_training_shape_feature(self, pca_comp = 29):

      images = np.sort(np.asarray(self.training_set_img_folder))
      landmarks = np.sort(np.asarray(self.training_set_landmarks_folder))
      frames_lmarks_XYord = get_landmarks_of_folder(landmarks)
      my_df = pd.DataFrame(frames_lmarks_XYord)
      intercalated_data = intercalate_data_of_datavectors(my_df, 0)
      my_dfc = pd.DataFrame(intercalated_data)

      my_landmarks = my_dfc
      pathi = self.aligned_training_images_path
      transformed_landmarks_total = align_and_transform_datavector_of_images(images[0], my_landmarks.iloc[0], my_landmarks, images, pathi)

      transformed_final = transformed_landmarks_total[0].flatten('F')
      for i in range (1, len(transformed_landmarks_total)):
        transformed_final = np.vstack((transformed_final, transformed_landmarks_total[i].flatten('F')))

      my_df = pd.DataFrame(transformed_final)
      my_df.to_csv(self.aligned_training_landmarks_path,header = False, index= False)

      ind = 0
      my_data_aligned_diff = []
      images_names = images
      current_neutral_index = 0
      for i in range(0, len(images_names)):
        if int(images_names[i].split('_')[2].split('.')[0]) == 1:
          current_neutral_index = i
          print('cambie neutral '+ str(current_neutral_index))
        my_data_aligned_diff.append(my_df.iloc[i]- my_df.iloc[current_neutral_index])
      my_data_aligned_diff = np.asarray(my_data_aligned_diff)

      my_data_forPCA = pd.DataFrame(my_data_aligned_diff)
      my_data_forPCA.to_csv(self.shape_diff_path,header = False, index= False)

      shape,pac= doPCA(my_data_forPCA, pca_comp)

      self.shape_pca = pac
      return shape
















In [ ]:
from glob import glob

path_of_used_series_landmarks = []
path_of_used_series_images = []
for i in np.sort(glob("/content/SynthesisEmotions/ck+copyAgainT/Landmarks/*/*/", recursive = True)):

    subject = i.split('/')[5]
    serie = i.split('/')[6]
    actual = subject + '/'+ serie
    path = '/content/SynthesisEmotions//ck+copyAgainT/Emotion_labels/Emotion/' +actual + '/'
    image_path = '/content/SynthesisEmotions/ck+copyAgainT/extended-cohn-kanade-images/cohn-kanade-images/' +actual+ '/'
    landmark_path = '/content/SynthesisEmotions/ck+copyAgainT/Landmarks/' + actual+ '/'

    filelabel = np.sort((os.listdir(path)))[0]
    longitud = np.sort((os.listdir(str(i))))
    imagelabel = np.sort(os.listdir(str(image_path)))
    landmarklabel = np.sort(os.listdir(str(landmark_path)))
    with open(path+ '/'+filelabel) as f:
        line = float(f.readline())
    if line == 4 or line == 6:
        path_of_used_series_landmarks.append(landmark_path + landmarklabel[0])
        path_of_used_series_landmarks.append(landmark_path + landmarklabel[len(longitud)-2])
        path_of_used_series_landmarks.append(landmark_path + landmarklabel[len(longitud)-1])
        path_of_used_series_images.append(image_path + imagelabel[0])
        path_of_used_series_images.append(image_path + imagelabel[len(longitud)-2])
        path_of_used_series_images.append(image_path + imagelabel[len(longitud)-1])
    else:
        path_of_used_series_landmarks.append(landmark_path + landmarklabel[0])
        path_of_used_series_landmarks.append(landmark_path + landmarklabel[len(longitud)-1])
        path_of_used_series_images.append(image_path + imagelabel[0])
        path_of_used_series_images.append(image_path + imagelabel[len(longitud)-1])

path_of_used_series_images

#vamos a hacer el pca con las neutrales y sin las neutrales.
path_of_used_series_landmarks

['/content/SynthesisEmotions/ck+copyAgainT/Landmarks/S010/002/S010_002_00000001_landmarks.txt',
 '/content/SynthesisEmotions/ck+copyAgainT/Landmarks/S010/002/S010_002_00000014_landmarks.txt',
 '/content/SynthesisEmotions/ck+copyAgainT/Landmarks/S010/004/S010_004_00000001_landmarks.txt',
 '/content/SynthesisEmotions/ck+copyAgainT/Landmarks/S010/004/S010_004_00000019_landmarks.txt',
 '/content/SynthesisEmotions/ck+copyAgainT/Landmarks/S010/006/S010_006_00000001_landmarks.txt',
 '/content/SynthesisEmotions/ck+copyAgainT/Landmarks/S010/006/S010_006_00000015_landmarks.txt',
 '/content/SynthesisEmotions/ck+copyAgainT/Landmarks/S011/001/S011_001_00000001_landmarks.txt',
 '/content/SynthesisEmotions/ck+copyAgainT/Landmarks/S011/001/S011_001_00000016_landmarks.txt',
 '/content/SynthesisEmotions/ck+copyAgainT/Landmarks/S011/002/S011_002_00000001_landmarks.txt',
 '/content/SynthesisEmotions/ck+copyAgainT/Landmarks/S011/002/S011_002_00000021_landmarks.txt',
 '/content/SynthesisEmotions/ck+copyAgai

In [ ]:
import glob

#training_img_set = np.sort(glob.glob('/content/SynthesisEmotions/ck+copyAgainT/extended-cohn-kanade-images/cohn-kanade-images/*/*/*',  recursive = True))
#training_lab_set = np.sort(glob.glob('/content/SynthesisEmotions/ck+copyAgainT/Landmarks/*/*/*',  recursive = True))

training_img_set =  path_of_used_series_images
training_lab_set = path_of_used_series_landmarks

480

In [ ]:

alig_im= '/content/SynthesisEmotions/probando_aligned'
alig_lan = '/content/SynthesisEmotions/probando_aligned_landmarks'
diff_path = '/content/SynthesisEmotions/probando_shapediff'


classi = EmotionClassifier(training_img_set, training_lab_set, alig_im, alig_lan, diff_path, 'hola', 'adios')

In [ ]:
shape = classi._get_training_shape_feature()

cambie neutral 0
cambie neutral 2
cambie neutral 4
cambie neutral 6
cambie neutral 8
cambie neutral 11
cambie neutral 14
cambie neutral 16
cambie neutral 18
cambie neutral 20
cambie neutral 22
cambie neutral 25
cambie neutral 27
cambie neutral 29
cambie neutral 31
cambie neutral 33
cambie neutral 35
cambie neutral 37
cambie neutral 40
cambie neutral 42
cambie neutral 44
cambie neutral 46
cambie neutral 48
cambie neutral 50
cambie neutral 53
cambie neutral 55
cambie neutral 57
cambie neutral 59
cambie neutral 61
cambie neutral 63
cambie neutral 65
cambie neutral 67
cambie neutral 69
cambie neutral 71
cambie neutral 73
cambie neutral 75
cambie neutral 77
cambie neutral 79
cambie neutral 81
cambie neutral 84
cambie neutral 86
cambie neutral 88
cambie neutral 90
cambie neutral 92
cambie neutral 94
cambie neutral 96
cambie neutral 98
cambie neutral 100
cambie neutral 102
cambie neutral 104
cambie neutral 107
cambie neutral 109
cambie neutral 111
cambie neutral 113
cambie neutral 115
cambie 

In [ ]:
shape.shape

(480, 29)

In [ ]:
classi._save_shape_pca()

In [ ]:
saved_spca = classi._load_shape_pca()


In [ ]:
(sum(saved_spca.explained_variance_ratio_))

0.9633423333577947